In [1]:
import numpy as np
import pandas as pd
np.random.seed(0)


#### Источник кода с комментариями : http://blog.ethanrosenthal.com/2016/01/09/explicit-matrix-factorization-sgd-als/

### Загружаем оценки и собираем из них матрицу оценок $r_{ij}$ 

In [2]:
# загружаем данные.
names = ['user_id', 'item_id', 'rating', 'timestamp']
df = pd.read_csv('C:\\Users\\wirze\\DataspellProjects\\recomend_hse\\week5\\ml-latest-small\\ratings.csv', sep=',',header=0, names=names)
n_users = df.user_id.unique().shape[0]
n_items = df.item_id.unique().shape[0]

movie_recode = {}
current_next_free_movie_id = 0
original_movie_id = []

# Создаем r_{ui} - нашу матрицу оценок
ratings = np.zeros((n_users, n_items))
for row in df.itertuples():
    if (row[2]) not in movie_recode:
        movie_recode[row[2]] = current_next_free_movie_id
        current_next_free_movie_id += 1
        original_movie_id.append(row[2])
    ratings[row[1]-1, movie_recode[row[2]]] = row[3]

In [7]:
n_users

610

In [8]:
n_items

9724

In [3]:
df.head()

,user_id,item_id,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [4]:
ratings[:10,:10]

array([[4. , 4. , 4. , 5. , 5. , 3. , 5. , 4. , 5. , 5. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 2. , 0. , 0. , 0. , 0. , 0. , 0. ],
       [4. , 0. , 0. , 0. , 4. , 0. , 0. , 4. , 0. , 0. ],
       [0. , 5. , 4. , 4. , 1. , 0. , 0. , 5. , 4. , 0. ],
       [4.5, 0. , 0. , 0. , 4.5, 0. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 4. , 5. , 0. , 0. , 3. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ]])

### Функция, которая делит множество оценок на test и train

In [9]:
# Делим данные на train и test 
# Для каждого пользователя 10 случайных оценок отправляем в test
def train_test_split(ratings):
    test = np.zeros(ratings.shape)
    train = ratings.copy()
    for user in np.arange(ratings.shape[0]):
        test_ratings = np.random.choice(ratings[user, :].nonzero()[0], 
                                        size=10, 
                                        replace=False)
        train[user, test_ratings] = 0.
        test[user, test_ratings] = ratings[user, test_ratings]
        
    # Проверяем, что   test и train не пересекаются
    assert(np.all((train * test) == 0)) 
    return train, test

### Метрика качества

In [10]:
from sklearn.metrics import mean_squared_error

def get_mse(pred, actual):
    pred = pred[actual.nonzero()].flatten()
    actual = actual[actual.nonzero()].flatten()
    return mean_squared_error(pred, actual)

### Класс, которые реализует алгоритм ALS и ExplicitMF c градиентным спуском и смещениями

In [11]:
from numpy.linalg import solve


class ExplicitMF():
    def __init__(self, 
                 ratings,
                 n_factors=40,
                 learning='sgd',
                 item_fact_reg=0.0, 
                 user_fact_reg=0.0,
                 item_bias_reg=0.0,
                 user_bias_reg=0.0,
                 verbose=False):
        """
        Train a matrix factorization model to predict empty 
        entries in a matrix. The terminology assumes a 
        ratings matrix which is ~ user x item
        
        Params
        ======
        ratings : (ndarray)
            User x Item matrix with corresponding ratings
        
        n_factors : (int)
            Number of latent factors to use in matrix 
            factorization model
        learning : (str)
            Method of optimization. Options include 
            'sgd' or 'als'.
        
        item_fact_reg : (float)
            Regularization term for item latent factors
        
        user_fact_reg : (float)
            Regularization term for user latent factors
            
        item_bias_reg : (float)
            Regularization term for item biases
        
        user_bias_reg : (float)
            Regularization term for user biases
        
        verbose : (bool)
            Whether or not to printout training progress
        """
        
        self.ratings = ratings
        self.n_users, self.n_items = ratings.shape
        self.n_factors = n_factors
        self.item_fact_reg = item_fact_reg
        self.user_fact_reg = user_fact_reg
        self.item_bias_reg = item_bias_reg
        self.user_bias_reg = user_bias_reg
        self.learning = learning
        if self.learning == 'sgd':
            self.sample_row, self.sample_col = self.ratings.nonzero()
            self.n_samples = len(self.sample_row)
        self._v = verbose

    def als_step(self,
                 latent_vectors,
                 fixed_vecs,
                 ratings,
                 _lambda,
                 type='user'):
        """
        One of the two ALS steps. Solve for the latent vectors
        specified by type.
        """
        if type == 'user':
            # Precompute
            YTY = fixed_vecs.T.dot(fixed_vecs)
            lambdaI = np.eye(YTY.shape[0]) * _lambda

            for u in range(latent_vectors.shape[0]):
                latent_vectors[u, :] = solve((YTY + lambdaI), 
                                             ratings[u, :].dot(fixed_vecs))
        elif type == 'item':
            # Precompute
            XTX = fixed_vecs.T.dot(fixed_vecs)
            lambdaI = np.eye(XTX.shape[0]) * _lambda
            
            for i in range(latent_vectors.shape[0]):
                latent_vectors[i, :] = solve((XTX + lambdaI), 
                                             ratings[:, i].T.dot(fixed_vecs))
        return latent_vectors

    def train(self, n_iter=10, learning_rate=0.1):
        """ Train model for n_iter iterations from scratch."""
        # initialize latent vectors        
        self.user_vecs = np.random.normal(scale=1./self.n_factors, size=(self.n_users, self.n_factors))
        self.item_vecs = np.random.normal(scale=1./self.n_factors, size=(self.n_items, self.n_factors))
        
        if self.learning == 'als':
            self.partial_train(n_iter,0)
        elif self.learning == 'sgd':
            self.learning_rate = learning_rate
            self.user_bias = np.zeros(self.n_users)
            self.item_bias = np.zeros(self.n_items)
            self.global_bias = np.mean(self.ratings[np.where(self.ratings != 0)])
            self.partial_train(n_iter,0)
    
    
    def partial_train(self, n_iter, iter_done):
        """ 
        Train model for n_iter iterations. Can be 
        called multiple times for further training.
        """
        ctr = 1
        while ctr <= n_iter:
            if (ctr+iter_done) % 10 == 0 and self._v:
                print (f'\tcurrent iteration: {ctr+iter_done}')
            if self.learning == 'als':
                self.user_vecs = self.als_step(self.user_vecs, 
                                               self.item_vecs, 
                                               self.ratings, 
                                               self.user_fact_reg, 
                                               type='user')
                self.item_vecs = self.als_step(self.item_vecs, 
                                               self.user_vecs, 
                                               self.ratings, 
                                               self.item_fact_reg, 
                                               type='item')
            elif self.learning == 'sgd':
                self.training_indices = np.arange(self.n_samples)
                np.random.shuffle(self.training_indices)
                self.sgd()
            ctr += 1

    def sgd(self):
        for idx in self.training_indices:
            u = self.sample_row[idx]
            i = self.sample_col[idx]
            prediction = self.predict(u, i)
            e = (self.ratings[u,i] - prediction) # error
            
            # Update biases
            self.user_bias[u] += self.learning_rate * (e - self.user_bias_reg * self.user_bias[u])
            self.item_bias[i] += self.learning_rate * (e - self.item_bias_reg * self.item_bias[i])
            
            #Update latent factors
            self.user_vecs[u, :] += self.learning_rate *\
                                    (e * self.item_vecs[i, :] - self.user_fact_reg * self.user_vecs[u,:])
            self.item_vecs[i, :] += self.learning_rate * \
                                    (e * self.user_vecs[u, :] - self.item_fact_reg * self.item_vecs[i,:])
    def predict(self, u, i):
        """ Single user and item prediction."""
        if self.learning == 'als':
            return self.user_vecs[u, :].dot(self.item_vecs[i, :].T)
        elif self.learning == 'sgd':
            prediction = self.global_bias + self.user_bias[u] + self.item_bias[i]
            prediction += self.user_vecs[u, :].dot(self.item_vecs[i, :].T)
            return prediction
    
    def predict_all(self):
        """ Predict ratings for every user and item."""
        predictions = np.zeros((self.user_vecs.shape[0], 
                                self.item_vecs.shape[0]))
        for u in range(self.user_vecs.shape[0]):
            for i in range(self.item_vecs.shape[0]):
                predictions[u, i] = self.predict(u, i)
                
        return predictions
    
    def calculate_learning_curve(self, iter_array, test, learning_rate=0.1):
        """
        Keep track of MSE as a function of training iterations.
        
        Params
        ======
        iter_array : (list)
            List of numbers of iterations to train for each step of 
            the learning curve. e.g. [1, 5, 10, 20]
        test : (2D ndarray)
            Testing dataset (assumed to be user x item).
        
        The function creates two new class attributes:
        
        train_mse : (list)
            Training data MSE values for each value of iter_array
        test_mse : (list)
            Test data MSE values for each value of iter_array
        """
        iter_array.sort()
        self.train_mse =[]
        self.test_mse = []
        iter_diff = 0
        for (i, n_iter) in enumerate(iter_array):
            if self._v:
                print (f'Iteration: {n_iter}')
            if i == 0:
                self.train(n_iter - iter_diff, learning_rate)
            else:
                self.partial_train(n_iter - iter_diff, iter_diff)

            predictions = self.predict_all()

            self.train_mse += [get_mse(predictions, self.ratings)]
            self.test_mse += [get_mse(predictions, test)]
            if self._v:
                print (f'MSE train:test: {round(self.train_mse[-1],2)} : {round(self.test_mse[-1],2)}\n')
            iter_diff = n_iter

#### для построения графиков

In [12]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

def plot_learning_curve(iter_array, model):
    plt.plot(iter_array, model.train_mse, \
             label='Training', linewidth=3)
    plt.plot(iter_array, model.test_mse, \
             label='Test', linewidth=3)


    plt.xticks(fontsize=16);
    plt.yticks(fontsize=16);
    plt.xlabel('Iterations', fontsize=25);
    plt.ylabel('MSE', fontsize=25);
    plt.legend(loc='best', fontsize=20);

##### Градиентный спуск с базовыми предикторами (biases)

In [13]:
np.random.seed(0)
train, test = train_test_split(ratings)
MF_SGD = ExplicitMF(train, 40, learning='sgd', verbose=True)
iter_array = [1, 2, 5, 10, 25, 50, 100]

In [14]:
%%time
MF_SGD.calculate_learning_curve(iter_array, test, learning_rate=0.001)

Iteration: 1
MSE train:test: 0.95 : 1.09

Iteration: 2
MSE train:test: 0.9 : 1.04

Iteration: 5
MSE train:test: 0.84 : 0.96

Iteration: 10
	current iteration: 10
MSE train:test: 0.8 : 0.91

Iteration: 25
	current iteration: 20
MSE train:test: 0.74 : 0.85

Iteration: 50
	current iteration: 30
	current iteration: 40
	current iteration: 50
MSE train:test: 0.7 : 0.83

Iteration: 100
	current iteration: 60
	current iteration: 70
	current iteration: 80
	current iteration: 90
	current iteration: 100
MSE train:test: 0.58 : 0.81

Wall time: 3min 20s


In [15]:
plot_learning_curve(iter_array, MF_SGD)

In [26]:
MF_SGD_10 = ExplicitMF(train, 10, learning='sgd', verbose=True)
iter_array = [1, 2, 5, 10, 25, 50, 100]

In [27]:
%%time
MF_SGD_10.calculate_learning_curve(iter_array, test, learning_rate=0.001)

Iteration: 1
MSE train:test: 0.95 : 1.09

Iteration: 2
MSE train:test: 0.91 : 1.04

Iteration: 5
MSE train:test: 0.84 : 0.97

Iteration: 10
	current iteration: 10
MSE train:test: 0.79 : 0.92

Iteration: 25
	current iteration: 20
MSE train:test: 0.74 : 0.87

Iteration: 50
	current iteration: 30
	current iteration: 40
	current iteration: 50
MSE train:test: 0.69 : 0.85

Iteration: 100
	current iteration: 60
	current iteration: 70
	current iteration: 80
	current iteration: 90
	current iteration: 100
MSE train:test: 0.58 : 0.84

Wall time: 4min 57s


In [28]:
MF_SGD_100 = ExplicitMF(train, 100, learning='sgd', verbose=True)
iter_array = [1, 2, 5, 10, 25, 50, 100]

In [29]:
%%time
MF_SGD_100.calculate_learning_curve(iter_array, test, learning_rate=0.001)

Iteration: 1
MSE train:test: 0.95 : 1.08

Iteration: 2
MSE train:test: 0.9 : 1.04

Iteration: 5
MSE train:test: 0.84 : 0.97

Iteration: 10
	current iteration: 10
MSE train:test: 0.8 : 0.91

Iteration: 25
	current iteration: 20
MSE train:test: 0.74 : 0.87

Iteration: 50
	current iteration: 30
	current iteration: 40
	current iteration: 50
MSE train:test: 0.71 : 0.85

Iteration: 100
	current iteration: 60
	current iteration: 70
	current iteration: 80
	current iteration: 90
	current iteration: 100
MSE train:test: 0.62 : 0.84

Wall time: 4min 58s


#### ALS без базовых предикторов

In [16]:
MF_ALS = ExplicitMF(train, 40, learning='als', verbose=True)
iter_array = [1, 2, 5, 10, 25, 50, 100]

In [17]:
%%time
MF_ALS.calculate_learning_curve(iter_array, test, learning_rate=0.001)

Iteration: 1
MSE train:test: 5.87 : 11.27

Iteration: 2
MSE train:test: 4.57 : 10.31

Iteration: 5
MSE train:test: 4.42 : 10.18

Iteration: 10
	current iteration: 10
MSE train:test: 4.41 : 10.14

Iteration: 25
	current iteration: 20
MSE train:test: 4.4 : 10.11

Iteration: 50
	current iteration: 30
	current iteration: 40
	current iteration: 50
MSE train:test: 4.4 : 10.11

Iteration: 100
	current iteration: 60
	current iteration: 70
	current iteration: 80
	current iteration: 90
	current iteration: 100
MSE train:test: 4.4 : 10.11

Wall time: 14min 48s


In [18]:

plot_learning_curve(iter_array, MF_ALS)

In [19]:
MF_SGD_hard_reg = ExplicitMF(train, 40, learning='sgd', verbose=True, item_bias_reg=100.0, user_bias_reg=100.0)
iter_array = [1, 2, 5, 10, 25, 50, 100]

In [20]:
%%time
MF_SGD_hard_reg.calculate_learning_curve(iter_array, test, learning_rate=0.001)

Iteration: 1
MSE train:test: 1.07 : 1.16

Iteration: 2
MSE train:test: 1.07 : 1.16

Iteration: 5
MSE train:test: 1.07 : 1.16

Iteration: 10
	current iteration: 10
MSE train:test: 1.07 : 1.16

Iteration: 25
	current iteration: 20
MSE train:test: 1.07 : 1.16

Iteration: 50
	current iteration: 30
	current iteration: 40
	current iteration: 50
MSE train:test: 1.02 : 1.15

Iteration: 100
	current iteration: 60
	current iteration: 70
	current iteration: 80
	current iteration: 90
	current iteration: 100
MSE train:test: 0.63 : 0.97

Wall time: 3min 21s


In [21]:
m = np.mean(ratings[np.where(ratings != 0)])
ratings_shifted = ratings - (ratings!=0)*m

In [22]:
ratings_shifted[:10, :10]

array([[ 0.49844302,  0.49844302,  0.49844302,  1.49844302,  1.49844302,
        -0.50155698,  1.49844302,  0.49844302,  1.49844302,  1.49844302],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , -1.50155698,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.49844302,  0.        ,  0.        ,  0.        ,  0.49844302,
         0.        ,  0.        ,  0.49844302,  0.        ,  0.        ],
       [ 0.        ,  1.49844302,  0.49844302,  0.49844302, -2.50155698,
         0.        ,  0.        ,  1.49844302,  0.49844302,  0.        ],
       [ 0.99844302,  0.        ,  0.        ,  0.        ,  0.99844302,
         0.        ,  0.        ,  0.        

In [23]:
m

3.501556983616962

In [52]:
test_s = np.zeros(ratings_shifted.shape)
train_s = ratings_shifted.copy()
non_zeros_index = np.nonzero(test)
test_s[non_zeros_index] = ratings_shifted[non_zeros_index]
train_s[non_zeros_index] = 0

In [46]:
np.random.seed(0)
train_s, test_s = train_test_split(ratings_shifted)
MF_SGD_s = ExplicitMF(train_s, 40, learning='sgd', verbose=True)
iter_array = [1, 2, 5, 10, 25, 50, 100]

In [53]:
for i in range(len(test[1,])):
    if test[1,i] > 0:
        print(test[1,i], test_s[1,i])

4.0 0.498443016383038
3.0 -0.501556983616962
4.0 0.498443016383038
4.5 0.998443016383038
4.0 0.498443016383038
3.0 -0.501556983616962
4.0 0.498443016383038
5.0 1.498443016383038
4.0 0.498443016383038
3.5 -0.0015569836169619933


In [54]:
for i in range(len(test_s[1,])):
    if test_s[1,i] != 0:
        print(test[1,i], test_s[1,i])

4.0 0.498443016383038
3.0 -0.501556983616962
4.0 0.498443016383038
4.5 0.998443016383038
4.0 0.498443016383038
3.0 -0.501556983616962
4.0 0.498443016383038
5.0 1.498443016383038
4.0 0.498443016383038
3.5 -0.0015569836169619933


In [56]:
MF_SGD_s = ExplicitMF(train_s, 40, learning='sgd', verbose=True)
iter_array = [1, 2, 5, 10, 25, 50, 100]

In [57]:
%%time
MF_SGD_s.calculate_learning_curve(iter_array, test_s, learning_rate=0.001)

Iteration: 1
MSE train:test: 0.95 : 1.08

Iteration: 2
MSE train:test: 0.9 : 1.04

Iteration: 5
MSE train:test: 0.84 : 0.97

Iteration: 10
	current iteration: 10
MSE train:test: 0.8 : 0.91

Iteration: 25
	current iteration: 20
MSE train:test: 0.74 : 0.87

Iteration: 50
	current iteration: 30
	current iteration: 40
	current iteration: 50
MSE train:test: 0.7 : 0.85

Iteration: 100
	current iteration: 60
	current iteration: 70
	current iteration: 80
	current iteration: 90
	current iteration: 100
MSE train:test: 0.58 : 0.84

Wall time: 5min 9s


In [58]:
MF_ALS = ExplicitMF(train_s, 40, learning='als', verbose=True)
iter_array = [1, 2, 5, 10, 25, 50, 100]

In [60]:
%%time
MF_ALS.calculate_learning_curve(iter_array, test_s, learning_rate=0.001)

Iteration: 1
MSE train:test: 0.63 : 1.11

Iteration: 2
MSE train:test: 0.5 : 1.06

Iteration: 5
MSE train:test: 0.49 : 1.05

Iteration: 10
	current iteration: 10
MSE train:test: 0.49 : 1.05

Iteration: 25
	current iteration: 20
MSE train:test: 0.49 : 1.05

Iteration: 50
	current iteration: 30
	current iteration: 40
	current iteration: 50
MSE train:test: 0.49 : 1.05

Iteration: 100
	current iteration: 60
	current iteration: 70
	current iteration: 80
	current iteration: 90
	current iteration: 100
MSE train:test: 0.49 : 1.05

Wall time: 8min 17s


Даже просто нормализация рейтингов на среднее значение по всем оценкам, позволило улучшить ALS, то есть оно чувствительно к bias. Тут (http://activisiongamescience.github.io/2016/01/11/Implicit-Recommender-Systems-Biased-Matrix-Factorization/) есть рецепт который позволит и для ALS учесть bias.